In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Initialization

In [2]:
import collections
import math
import os.path
import random
import time
import traceback
from multiprocessing import Process, Lock

import json
import pandas as pd
import requests

I place the API keys I use inside api_keys.txt to keep them private.

In [4]:
with open('api_keys.txt', 'r') as f:
    api_keys = json.load(f)
    riot_api_key = api_keys['riot-api-key']
    championgg_api_key = api_keys['champion-gg-api-key']

## API call helper methods

In [5]:
def make_request(url, **kwargs):
    '''
    Make API call to url, with the given parameters in **kwargs
    
    Parameters
    ----------
    url : str
        Full url used to send request
    **kwargs
        Additional keyword args to pass to `requests.get()`
        
    Returns
    -------
    dict
        JSON from API call converted to a dict
    '''
    r = requests.get(url, **kwargs)
    # Rate limited, wait and redo request
    if r.status_code == 429:
        retry_after = int(r.headers['Retry-After'])
        print(f"Rate limited for request {url}, sleeping for {retry_after} s")
        time.sleep(retry_after)
        return make_request(url, **kwargs)
    elif r.status_code >= 500:
        print(f"Server error {r.status_code}, retrying in 60 s")
        time.sleep(60)
        return make_request(url, **kwargs)
    elif r.status_code != 200:
        print(f"got {r.status_code} error when making request {url}")
        return
    return r.json()


def make_riot_request(uri, region, params=None):
    '''
    Make an call to Riot's API.
    
    Parameters
    ----------
    uri : str
        The desired URI
    region : str
        The desired region (NA1, EUW1, KR, etc.)
    params : dict, optional
        Additional params to append to the GET request
        
    Returns
    -------
    dict
        JSON from API call converted to a dict
    '''
    return make_request(f"https://{region}.api.riotgames.com{uri}", headers={'X-Riot-Token': riot_api_key}, 
                        params=params)


def make_championgg_request(uri, params={}):
    '''
    Make an call to Champion.gg's API.
    
    Parameters
    ----------
    uri : str
        The desired URI
    params : dict, optional
        Additional params to append to the GET request
        
    Returns
    -------
    dict
        JSON from API call converted to a dict
    '''
    params['api_key'] = championgg_api_key  # Add API key to params
    return make_request(f"http://api.champion.gg/v2{uri}", params=params)

## Static data extraction

Create methods to extract static data. Since rate limit for static data is so low, we retrieve it as little as possible.

In [6]:
# Get mapping from summoner spell id to name
spell_id_to_name = make_riot_request('/lol/static-data/v3/summoner-spells', 'NA1', params={'dataById': 'true'})['data']
spell_id_to_name = {int(k): v['name'] for k, v in spell_id_to_name.items()}

In [7]:
# Get mapping from champion id to name
champ_id_to_name = make_riot_request('/lol/static-data/v3/champions', 'NA1', params={'dataById': 'true'})['data']
champ_id_to_name = {int(k): v['name'] for k, v in champ_id_to_name.items()}

In [8]:
# Get number of most recent patch
PATCH = make_riot_request('/lol/static-data/v3/versions', 'NA1')[0]

In [9]:
# Set other game constants
SEASON = 11  # Season 8
RANKED_QUEUE = 420
NORMAL_QUEUE = 400  # Normal draft

# Data Collection

## Helper methods

In [10]:
def is_championgg_tier(tier):
    '''Return True if tier is not diamond, master, or challenger, False otherwise'''
    return tier not in ['DIAMOND', 'MASTER', 'CHALLENGER']

def rank_to_mmr(tier, rank, lp):
    '''
    Convert ranks to continuous scale.
    
    Parameters
    ----------
    tier : str
        Major division (BRONZE, SILVER, GOLD, PLATINUM, DIAMOND, MASTER, CHALLENGER)
    rank : str
        Minor divison (V, IV, III, II, I)
    lp : int
        League points as value from 0-100 (except for Masters+, where lp has no limit)
    
    Returns
    -------
    int
        Number that represents "mmr". Ranges from 0-2500 for ranks under Masters, and 2500+ for Masters+.
    '''
    tier = tier if tier != 'CHALLENGER' else 'MASTER'
    rank = rank if tier != 'CHALLENGER' and tier != 'MASTER' else 'V'
    tier_to_num = {'BRONZE': 0, 'SILVER': 1, 'GOLD': 2, 'PLATINUM': 3, 'DIAMOND': 4, 'MASTER': 5}
    rank_to_tier = {'V': 0, 'IV': 1, 'III': 2, 'II': 3, 'I': 4}
    return tier_to_num[tier] * 500 + rank_to_tier[rank] * 100 + lp


def get_roles(participants, match_id, region):
    '''
    Manually determine which player is in which role, since Riot API's automatic labeling is not very accurate.

    Parameters
    ----------
    x : list
        List of participants as given by Riot API when querying match
    match_id : int
        ID of desired match
    region : str
        Region to query match from
        
    Returns
    -------
    dict
        Mapping of participant ids to their role (TOP, JUNGLE, MIDDLE, DUO_CARRY, DUO_SUPPORT)
    '''
    roles = {}
    unique_roles = set()
    team_100 = {p['participantId']: p for p in participants if p['teamId'] == 100}
    team_200 = {p['participantId']: p for p in participants if p['teamId'] == 200}
    match_frames = make_riot_request(f"/lol/match/v3/timelines/by-match/{match_id}", region)['frames']
    for team_id, team in {'100': team_100, '200': team_200}.items():
        # Find junglers by seeing who has smite summoner spell
        role = 'JUNGLE'
        unique_role = role + team_id
        for pid, p in team.items():
            if spell_id_to_name[p['spell1Id']] == 'Smite' or spell_id_to_name[p['spell2Id']] == 'Smite':
                if unique_role in unique_roles:  # Return if more than 1 smite on a team
                    print('More than one person has smite on a team')
                    return
                roles[pid] = role
                unique_roles.add(unique_role)
        if unique_role not in unique_roles:  # Return if no smite on a team
            print('No smite on a team')
            return
        
        # Label support as summoner with least cs
        support_id = min(team.values(), key=lambda p: p['stats']['totalMinionsKilled'] + 
                         p['stats']['neutralMinionsKilled'])['participantId']
        if support_id in roles:  # Make sure jungle isn't relabeled as support
            print('Jungle got relabeled as support')
            return
        role = 'DUO_SUPPORT'
        roles[support_id] = role
        unique_roles.add(role + team_id)
        
        # Label top, mid, and adc based on position by level 2+
        lane_pos = {'TOP': (2000, 14881 - 2000), 'MIDDLE': (14820 / 2, 14881 / 2), 'DUO_CARRY': (14820 - 2000, 2000)}
        num_assigned = 0
        for frame in match_frames:
            for pid, p in {int(k): v for k, v in frame['participantFrames'].items()}.items():
                if pid in team and p['level'] >= 2 and pid not in roles:
                    if 'position' not in p:
                        continue
                    dists = {lane: math.hypot(p['position']['x'] - pos[0], p['position']['y'] - pos[1]) 
                             for lane, pos in lane_pos.items()}  # Distance between summoner and center of lane
                    if min(dists.values()) > 2000:  # Continue if too far away from center of lane
                        continue
                    role = min(dists, key=dists.get)  # Chose lane that summoner is closest to
                    unique_role = role + team_id
#                     print(p['position']['x'], p['position']['y'])
#                     print(dists)
                    if unique_role in unique_roles:  # Return if role is repeated
                        print(f"Role {role} repeated on a team")
#                         print([champ_id_to_name[participants[int(i) - 1]['championId']] for i in roles])
#                         print(champ_id_to_name[participants[pid - 1]['championId']])
                        return
                    roles[pid] = role
                    unique_roles.add(unique_role)
                    num_assigned += 1
            if num_assigned == 3:
                break
        if num_assigned != 3:  # Return if someone didn't reach lvl 2 (such as disconnect)
            print('Someone did not reach lvl 2 in the game')
            return
        
    return roles


def capitalize(s):
    '''Capitalize first letter in a word'''
    return s[0].capitalize() + s[1:]


def store_championgg_stats(summoner, role_info, champ_stats, blank=False):
    '''
    Transfer general champion stats, positions, and previous positions in role from champion.gg to summoner
    
    Parameters
    ----------
    summoner : dict
        Summoner that will store the champion stats
    role_info : dict
        Dictionary of general stats for a champion in a role
    champ_stats : list
        Champion stats that are being stored
    blank : bool, optional
        Whether to fill stats as N/A
    '''
    for stat in champ_stats:
        prefix = 'champion' + capitalize(stat)
        suffix = 's' if stat != 'overallPerformanceScore' else ''
        if stat != 'overallPerformanceScore':
            summoner[prefix] = role_info[stat] if not blank else 'N/A'
        summoner[prefix + 'Position'] = role_info['positions'][stat + suffix] if not blank else 'N/A'
        if 'previous' + capitalize(stat) + suffix not in role_info['positions'] or blank:
            summoner[prefix + 'PreviousPosition'] = 'N/A'
        else:
            summoner[prefix + 'PreviousPosition'] = role_info['positions']['previous' + capitalize(stat) + suffix]


def store_cdragon_stats(summoner, champ_stats):
    '''
    Transfer general champion stats from CommunityDragon to summoner
    
    Parameters
    ----------
    summoner : dict
        Summoner that will store the champion stats
    champ_stats : dict
        Champion stats that are being stored
    '''
    for k, v in champ_stats.items():
        summoner['champion' + capitalize(k)] = v  # Capitalize first letter of stat for camelCase
        

def store_matchup(summoner_matchup, role):
    '''
    Transfer winrates in the lane matchup to the summoners in the matchup
    
    Parameters
    ----------
    summoner_matchup : list
        List of the two summoners in the matchup
    role : str
        Role for the matchup
    
    Returns
    -------
    bool
        True if we successfully found the matchup through champion.gg, False otherwise
    '''
    champ_ids = [summoner['championId'] for summoner in summoner_matchup]
    for summoner, champ_id in zip(summoner_matchup, champ_ids):
        params = {'elo': summoner['tier']} if is_championgg_tier(summoner['tier']) else {}
        params['limit'] = 500  # High limit b/c of synergy and adc/support matchups
        matchup_data = make_championgg_request(f"/champions/{champ_id}/{role}/matchups", params=params)
        for matchup in matchup_data:
            # Continue if too little data, not direct matchup (synergy, etc.), or desired champs are not included
            if (matchup['count'] < 10 or matchup['role'] != role
                    or not all(champ_id in matchup['_id'].values() for champ_id in champ_ids)):
                continue
            for summoner in summoner_matchup:
                matchup_champ_data = (matchup['champ1'] if summoner['championId'] == matchup['champ1_id']
                    else matchup['champ2'])
                summoner['matchupWinRate'] = matchup_champ_data['winrate']
            return True
    return False

##  Feature extraction from match

In [11]:
def get_match(match_id, region):
    '''
    Retrieve data for all 10 summoners given a match.
    
    Parameters
    ----------
    match_id : int
        Match id to query
    region : str
        Region to query from
        
    Returns
    -------
    OrderedDict
        Return dictionary of features described below
        We store the following stats for convenience (removed later):
        - summonerId
        - accountId
        - summoner name
        - champion name
        We store the following general stats per summoner from Riot API:
        - role (TOP, JUNGLE, MIDDLE, DUO_CARRY, DUO_SUPPORT)
        - team id (100 for blue or 200 for red)
        - champion id
        - mastery points for champion
        - whether summoner is on "hot streak"
        - whether summoner is "veteran" in league (100+ games played)
        - whether summoner is "fresh blood" (new to league)
        - summoner winrate in ranked
        - total number of ranked games
        - tier (BRONZE, SILVER, GOLD, PLATINUM, DIAMOND, MASTER, CHALLENGER)
        - mmr (from 0 to 2500+)
        - hours between match and the previous ranked or normal match
        - percent of games played as same role in last 10 games
        - percent of games played as same role in last 50 games
        - percent of games that were ranked in last 50 games
        - percent of games that were played with the same champion in last 50 games
        - percent of games that were played in ranked with the same champion in last 50 games
        We store the following champion.gg stats, as well as their position and previous position within role:
        - champion ban rate
        - champion win rate in role (may be N/A)
        - champion play rate in role (may be N/A)
        We store the following additional champion.gg stats:
        - champion.gg's performance score for the champion in role (may be N/A)
        - champion.gg's previous performance score for the champion in role (may be N/A)
        - champion percent played in role (may be N/A)
        - winrate for lane matchup (may be N/A)
        We store the following general champion stats from CommunityDragon:
        - champion "style" as a scale, with 1 using autoattacks only and 10 using abilities only
        - champion difficulty on scale of 1 to 3
        - champion main damage type (physical or magic)
        - champion damage on scale of 1 to 3
        - champion durability on scale of 1 to 3
        - champion crowd control on scale of 1 to 3
        - champion mobility on scale of 1 to 3
        - champion utility on scale of 1 to 3
    '''
    # Hope that Riot API's automatic role labeling is correct
    get_match_role = lambda match: match['lane'] if match['lane'] != 'BOTTOM' else match['role']
    mmr_to_tier = lambda mmr: {0: 'BRONZE', 1: 'SILVER', 2: 'GOLD', 3: 'PLATINUM', 4: 'DIAMOND', 5: 'MASTER'}[mmr // 500]
    
    match_content = make_riot_request(f"/lol/match/v3/matches/{match_id}", region)
    if not match_content:
        return
    # Must be current patch
    if PATCH[:-2] not in match_content['gameVersion']:
        print('Not current patch')
        return
    # Must be Ranked Solo/Duo game
    if match_content['queueId'] != RANKED_QUEUE:
        print('Not a ranked solo/duo game')
        return
    
    # Store very basic data needed for other API calls
    data = collections.defaultdict(collections.OrderedDict)
    for p in match_content['participantIdentities']:
        summoner = data[p['participantId']]
        summoner['summonerId'] = p['player']['summonerId']
        summoner['accountId'] = p['player']['currentAccountId']
        summoner['summonerName'] = p['player']['summonerName']
    for p in match_content['participants']:
        summoner = data[p['participantId']]
        summoner['teamId'] = p['teamId']
        summoner['championId'] = p['championId']
        summoner['championName'] = champ_id_to_name[summoner['championId']]
    
    # Store roles for each player
    roles = get_roles(match_content['participants'], match_id, region)
    if not roles:
        print('Could not determine roles for players')
        return
    for pid, summoner in data.items():
        summoner['role'] = roles[pid]
        
    # Store ranked stats
    unranked = collections.defaultdict(list)  # Keep track of unranked players
    team_mmr = collections.defaultdict(int)
    for summoner in data.values():
        ranked_content = make_riot_request(f"/lol/league/v3/positions/by-summoner/{summoner['summonerId']}", region)
        if not ranked_content:
            unranked[summoner['teamId']].append(summoner)
            continue
        found_queue = False
        for queue in ranked_content:
            if queue['queueType'] == 'RANKED_SOLO_5x5':
                summoner['hotStreak'] = queue['hotStreak']
                summoner['veteran'] = queue['veteran']
                summoner['freshBlood'] = queue['freshBlood']
                summoner['summonerWinRate'] = queue['wins'] / (queue['wins'] + queue['losses'])
                summoner['totalRankedGames'] = queue['wins'] + queue['losses']
                summoner['tier'] = queue['tier']
                summoner['mmr'] = rank_to_mmr(queue['tier'], queue['rank'], queue['leaguePoints'])
                team_mmr[summoner['teamId']] += summoner['mmr']
                found_queue = True
                break
        if not found_queue:
            unranked[summoner['teamId']].append(summoner)
    # Assign average team mmr to unranked summoners
    for team, total_mmr in team_mmr.items():
        avg_mmr = total_mmr / (5 - len(unranked[team]))
        for summoner in unranked[team]:
            summoner['hotStreak'] = False
            summoner['veteran'] = False
            summoner['freshBlood'] = True
            summoner['summonerWinRate'] = 0.5
            summoner['totalRankedGames'] = 0
            summoner['tier'] = mmr_to_tier(avg_mmr)
            summoner['mmr'] = avg_mmr
    
    for summoner in data.values():
        # Store mastery points of given champion
        mastery_content = make_riot_request(f"/lol/champion-mastery/v3/champion-masteries/by-summoner/"
                                            f"{summoner['summonerId']}/by-champion/{summoner['championId']}", region)
        if mastery_content:
            summoner['masteryPoints'] = mastery_content['championPoints']
        else:
            summoner['masteryPoints'] = 0
                                
        # Store data from the summoner's previous matches
        match_list = make_riot_request(f"/lol/match/v3/matchlists/by-account/{summoner['accountId']}", region, 
                                       params={'season': SEASON, 'queue': [RANKED_QUEUE, NORMAL_QUEUE], 'endIndex': 50})['matches']
        # Make sure we only consider games before the desired game
        for i, match in enumerate(match_list):
            if match['gameId'] == match_content['gameId']:
                match_list = match_list[(i+1):]
        if len(match_list) == 0:
            print(f"No previous matches for this season for summoner {summoner['summonerName']}")
            return
        
        summoner['hoursSincePreviousGame'] = (match_content['gameCreation'] - match_list[0]['timestamp']) / (1000 * 60 * 60)
        is_same_role = [get_match_role(match) == summoner['role'] for match in match_list]
        summoner['percentRolePlayedLast10'] = sum(is_same_role[:10]) / len(is_same_role[:10])
        summoner['percentRolePlayedLast50'] = sum(is_same_role) / len(is_same_role)
        is_ranked_match = [match['queue'] == RANKED_QUEUE for match in match_list]
        summoner['percentRankedGamesPlayed'] = sum(is_ranked_match) / len(is_ranked_match)
        is_game_as_champ = [match['champion'] == summoner['championId'] for match in match_list]
        summoner['percentGamesWithChampion'] = sum(is_game_as_champ) / len(is_game_as_champ)
        is_ranked_game_as_champ = [x and y for x, y in zip(is_ranked_match, is_game_as_champ)]
        summoner['percentRankedGamesWithChampion'] = sum(is_ranked_game_as_champ) / len(is_ranked_game_as_champ)
        
        previous_game = make_riot_request(f"/lol/match/v3/matches/{match_list[0]['gameId']}", region)
        for prev_p in previous_game['participantIdentities']:
            if prev_p['player']['summonerId'] == summoner['summonerId']:
                pid = prev_p['participantId']
                summoner['previousGameWon'] = previous_game['participants'][prev_p['participantId'] - 1]['stats']['win']
        
        # Store general champion stats from champion.gg
        params = {'champData': 'positions'}
        if is_championgg_tier(summoner['tier']):
            params['elo'] = summoner['tier']
        champ_content_championgg = make_championgg_request(f"/champions/{summoner['championId']}", params=params)
        
        store_championgg_stats(summoner, champ_content_championgg[0], ['banRate'])
        # If we can't get stats for champion in that role, assign as N/A
        found_role = False
        for role_info in champ_content_championgg:
            if role_info['role'] == summoner['role']:
                store_championgg_stats(summoner, role_info, ['winRate', 'playRate', 'overallPerformanceScore'])
                summoner['championPercentRolePlayed'] = role_info['percentRolePlayed']
                found_role = True
                break
        if not found_role:
            store_championgg_stats(summoner, role_info, ['winRate', 'playRate', 'overallPerformanceScore'], blank=True)
            summoner['championPercentRolePlayed'] = 'N/A'
                
        # Store more general champion stats from CommunityDragon
        champ_content_cdragon = make_request(f"http://cdn.communitydragon.org/{PATCH}/champion/"
                                             f"{summoner['championId']}/data")
        # Store style, difficulty, and damage type
        store_cdragon_stats(summoner, champ_content_cdragon['tacticalInfo'])
        # Store damage, durability, crowd control, mobility, and utility
        store_cdragon_stats(summoner, champ_content_cdragon['playstyleInfo'])
                
    # Store winrates for lane matchups
    for role in ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']:
        summoner_matchup = [summoner for summoner in data.values() if summoner['role'] == role]
        found_matchup = store_matchup(summoner_matchup, role)
        if not found_matchup:
            for summoner in summoner_matchup:
                summoner['matchupWinRate'] = 'N/A'
#             print(f"Could not find matchup between {champ_id_to_name[summoner_matchup[0]['championId']]} and "
#                   f"{champ_id_to_name[summoner_matchup[1]['championId']]} in role {role}")
                        
    return data

## Obtain list of league ids

We want our sample of matches to accurately match the distribution of ranks. Therefore, we use a list of league ids compiled by canisback, which is close enough to the real distribution.

In [11]:
for region in ['euw1', 'kr', 'na1']:
    file_name = f"league_{region}.csv"
    if not os.path.isfile(file_name):
        r = requests.get(f"http://canisback.com/leagueId/{file_name}.csv")
        with open(file_name, 'wb') as f:  
            f.write(r.content)

## Data collection

In [18]:
def collect_data(region, lock):
    '''Collect matches from a region, and store in a csv'''
    skip_rows = 40
    region_df = pd.read_csv(f"league_{region}.csv", skiprows=range(1, skip_rows))[['leagueId', 'tier']]
    total_queries = 0
    total_games = 0
    for i, row in region_df.iterrows():
        print(f"Querying league #{i} from region {region}")
        league = make_riot_request(f"/lol/league/v3/leagues/{row['leagueId']}", region)['entries']
        for summoner in league:
            total_queries += 1
            # Select most recent game played by summoner
            account_id = make_riot_request(f"/lol/summoner/v3/summoners/{summoner['playerOrTeamId']}", region)['accountId']
            match_list = make_riot_request(f"/lol/match/v3/matchlists/by-account/{account_id}", region, 
                                          params={'queue': RANKED_QUEUE, 'season': SEASON, 'endIndex': 1})['matches']
            if len(match_list) == 0:
                continue
            try:
                match_data = get_match(match_list[0]['gameId'], region)
            except Exception as e:
                traceback.print_exc()
                continue
            if not match_data:
                continue
            
            # Make sure we have columns in exact same order
            # Also split each match into 2 datapoints, with teams swapped
            match_data = match_data.values()
            ally100 = pd.Series()
            ally200 = pd.Series()
            for team_id, team_data in {100: ally100, 200: ally200}.items():
                ally_team = [summoner for summoner in match_data if summoner['teamId'] == team_id]
                enemy_team = [summoner for summoner in match_data if summoner['teamId'] != team_id]
                for role in ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']:
                    summoner = [summoner for summoner in ally_team if summoner['role'] == role][0]
                    for k, v in summoner.items():
                        team_data['ally' + role + k] = v
                for role in ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']:
                    summoner = [summoner for summoner in enemy_team if summoner['role'] == role][0]
                    for k, v in summoner.items():
                        team_data['enemy' + role + k] = v
            ally100['region'] = region
            ally200['region'] = region
            
            df = pd.DataFrame(columns=ally100.index)
            df = df.append(ally100, ignore_index=True)
            df = df.append(ally200, ignore_index=True)
                        
            # Write to file
            lock.acquire()
            data_file = 'summoner_data.csv'
            header = False
            if not os.path.isfile(data_file):
                header = True
            elif os.path.getsize(data_file) > 10**9:  # Don't let file exceed 1 GB
                return
            with open(data_file, 'a') as f:
                df.to_csv(f, index=False, header=header)
            lock.release()
            
            total_games += 1
            print(f"Collected {total_games} games from region {region}")
            print(f"Queried {total_queries} games from region {region}")
    print(f"Queried all possible games from region {region}")

Query matches from EUW, KR, and NA at the same time.

In [17]:
data_collect_flag = False
if data_collect_flag:
    lock = Lock()
    for region in ['euw1', 'kr', 'na1']:
        Process(target=collect_data, args=(region, lock)).start()

Querying league #0 from region euw1
Querying league #0 from region na1
Querying league #0 from region kr
Not current patch
Not current patch
Not current patch
Collected 1 games from region na1
Queried 1 games from region na1
Collected 1 games from region euw1
Queried 4 games from region euw1
Collected 2 games from region na1
Queried 2 games from region na1
Collected 1 games from region kr
Queried 1 games from region kr
Not current patch
Not current patch
Not current patch
Not current patch
Rate limited for request https://na1.api.riotgames.com/lol/match/v3/matches/2720962923, sleeping for 68 s
Not current patch
No previous matches for this season for summoner EC EnglishCorner
Not current patch
Not current patch
Rate limited for request https://euw1.api.riotgames.com/lol/summoner/v3/summoners/36786908, sleeping for 54 s
Collected 2 games from region kr
Queried 4 games from region kr
Rate limited for request https://kr.api.riotgames.com/lol/league/v3/positions/by-summoner/50711615, sleep

Collected 19 games from region kr
Queried 34 games from region kr
Collected 18 games from region euw1
Queried 48 games from region euw1
Not current patch
Not current patch
Not current patch
Role DUO_CARRY repeated on a team
Could not determine roles for players
Not current patch
Not current patch
Not current patch
Jungle got relabeled as support
Could not determine roles for players
Not current patch
Collected 20 games from region na1
Queried 54 games from region na1
Not current patch
Not current patch
More than one person has smite on a team
Could not determine roles for players
Not current patch
Not current patch
Not current patch
Not current patch
Collected 20 games from region kr
Queried 35 games from region kr
Not current patch
Not current patch
Rate limited for request https://na1.api.riotgames.com/lol/match/v3/matches/2807610084, sleeping for 69 s
Collected 19 games from region euw1
Queried 58 games from region euw1
Not current patch
Not current patch
Role DUO_CARRY repeated on 

Could not determine roles for players
Not current patch
Not current patch
Not current patch
Rate limited for request https://euw1.api.riotgames.com/lol/match/v3/matchlists/by-account/235351794, sleeping for 50 s
Collected 35 games from region kr
Queried 73 games from region kr
Not current patch
Not current patch
Rate limited for request https://kr.api.riotgames.com/lol/league/v3/positions/by-summoner/19623613, sleeping for 14 s
Collected 33 games from region euw1
Queried 108 games from region euw1
Collected 36 games from region kr
Queried 76 games from region kr
No previous matches for this season for summoner BaekSeol
Not current patch
Not current patch
Not current patch
Collected 34 games from region euw1
Queried 109 games from region euw1
Not current patch
Collected 37 games from region kr
Queried 77 games from region kr
Rate limited for request https://euw1.api.riotgames.com/lol/match/v3/matchlists/by-account/225948304, sleeping for 29 s
Not current patch
Collected 34 games from re

Collected 47 games from region na1
Queried 165 games from region na1
Role DUO_CARRY repeated on a team
Could not determine roles for players
Collected 52 games from region kr
Queried 106 games from region kr
Not current patch
Not current patch
Not current patch
Not current patch
Rate limited for request https://kr.api.riotgames.com/lol/league/v3/positions/by-summoner/66661002, sleeping for 10 s
Rate limited for request https://na1.api.riotgames.com/lol/champion-mastery/v3/champion-masteries/by-summoner/31379228/by-champion/81, sleeping for 47 s
Collected 53 games from region kr
Queried 108 games from region kr
Collected 47 games from region euw1
Queried 165 games from region euw1
Collected 48 games from region na1
Queried 170 games from region na1
Collected 48 games from region euw1
Queried 166 games from region euw1
Collected 49 games from region na1
Queried 171 games from region na1
Not current patch
Not current patch
No smite on a team
Could not determine roles for players
Someone d

Querying league #1 from region euw1
Collected 70 games from region kr
Queried 131 games from region kr
Not current patch
Collected 63 games from region euw1
Queried 201 games from region euw1
Rate limited for request https://euw1.api.riotgames.com/lol/match/v3/matches/3673096310, sleeping for 48 s
Collected 64 games from region na1
Queried 226 games from region na1
Not current patch
Collected 71 games from region kr
Queried 133 games from region kr
Collected 65 games from region na1
Queried 228 games from region na1
Not current patch
Not current patch
Not current patch
Rate limited for request https://kr.api.riotgames.com/lol/champion-mastery/v3/champion-masteries/by-summoner/1391376/by-champion/63, sleeping for 18 s
Rate limited for request https://na1.api.riotgames.com/lol/league/v3/positions/by-summoner/56863043, sleeping for 70 s
Collected 64 games from region euw1
Queried 202 games from region euw1
Not current patch
Collected 72 games from region kr
Queried 134 games from region k

...

Collected 510 games from region euw1
Queried 1561 games from region euw1
Not current patch
Collected 529 games from region kr
Queried 1221 games from region kr
Collected 511 games from region euw1
Queried 1563 games from region euw1
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Collected 530 games from region kr
Queried 1222 games from region kr
Not current patch
Not current patch
Collected 506 games from region na1
Queried 1710 games from region na1
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Collected 512 games from region euw1
Queried 1573 games from region euw1
Not current patch
Rate limited for request https://euw1.api.riotgames.com/lol/league/v3/positions/by-summoner/107977002, s

Jungle got relabeled as support
Could not determine roles for players
Not current patch
Not current patch
Rate limited for request https://kr.api.riotgames.com/lol/match/v3/matchlists/by-account/5904602, sleeping for 18 s
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Collected 546 games from region kr
Queried 1260 games from region kr
Not current patch
Collected 520 games from region na1
Queried 1780 games from region na1
Collected 527 games from region euw1
Queried 1614 games from region euw1
Not current patch
Collected 521 games from region na1
Queried 1781 games from region na1
Not current patch
Not current patch
Not current patch
Rate limited for request https://euw1.api.riotgames.com/lol/match/v3/matchlists/by-account/218232196, sleeping for 50 s
Collected 547 games from region kr
Queried 1262 games from region kr
Not current patch
Rate limited for request https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/217110657, sleepi

Queried 1299 games from region kr
Not current patch
Rate limited for request https://kr.api.riotgames.com/lol/champion-mastery/v3/champion-masteries/by-summoner/69150791/by-champion/21, sleeping for 19 s
Collected 540 games from region euw1
Queried 1660 games from region euw1
Collected 535 games from region na1
Queried 1834 games from region na1
Collected 561 games from region kr
Queried 1301 games from region kr
Not current patch
Collected 541 games from region euw1
Queried 1661 games from region euw1
Not current patch
Not current patch
Not current patch
Collected 536 games from region na1
Queried 1835 games from region na1
Server error 503, retrying in 60 s
Collected 537 games from region na1
Queried 1836 games from region na1
Rate limited for request https://na1.api.riotgames.com/lol/league/v3/positions/by-summoner/21451900, sleeping for 66 s
Collected 562 games from region kr
Queried 1306 games from region kr
Not current patch
Role DUO_CARRY repeated on a team
Could not determine r

Rate limited for request https://kr.api.riotgames.com/lol/match/v3/matches/3244880247, sleeping for 21 s
Collected 556 games from region euw1
Queried 1705 games from region euw1
Not current patch
Not current patch
Not current patch
Collected 551 games from region na1
Queried 1882 games from region na1
Not current patch
Not current patch
Not current patch
Not current patch
Not current patch
Collected 578 games from region kr
Queried 1339 games from region kr
Collected 557 games from region euw1
Queried 1709 games from region euw1
Not current patch
Not current patch
Role DUO_CARRY repeated on a team
Could not determine roles for players
Not current patch
Collected 552 games from region na1
Queried 1888 games from region na1
Not current patch
Rate limited for request https://euw1.api.riotgames.com/lol/match/v3/matchlists/by-account/226046231, sleeping for 53 s
Rate limited for request https://na1.api.riotgames.com/lol/champion-mastery/v3/champion-masteries/by-summoner/37143322/by-champion

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Storing win/loss for each match

I accidentally forgot to store whether the match was won or lost, so I have to go back to do that. I didn't store the match id, so I'll have to find the match based on its time to the previous match, since I did store that.

In [79]:
for i, row in data.iterrows():
    if i % 2 == 1:
        continue
    account_id = row['allyTOPaccountId']
    try:
        match_list = make_riot_request(f"/lol/match/v3/matchlists/by-account/{account_id}", row['region'], 
                                      params={'season': SEASON, 'queue': [RANKED_QUEUE, NORMAL_QUEUE]})['matches']
    except Exception as e:
        traceback.print_exc()
        print(f"Error when fetching matchlist for row {i}")
        continue
    found_match = False
#     print(row[['allyTOPaccountId', 'allyTOPsummonerName', 'allyTOPchampionId', 'allyJUNGLEsummonerName']])
#     print(champ_id_to_name[row['allyTOPchampionId']])
    for match_idx in range(len(match_list) - 1):
        hours_between = (match_list[match_idx]['timestamp'] - match_list[match_idx + 1]['timestamp']) / (1000 * 60 * 60)
#         print(hours_between, row['allyTOPhoursSincePreviousGame'])
        if abs(row['allyTOPhoursSincePreviousGame'] - hours_between) < 0.001:
            try:
                match = make_riot_request(f"/lol/match/v3/matches/{match_list[match_idx]['gameId']}", row['region'])
            except Exception as e:
                traceback.print_exc()
                print(f"Could not fetch match for row {i}")
                break
            for p in match['participantIdentities']:
                if p['player']['accountId'] == row['allyTOPaccountId']:
                    team = 0 if p['participantId'] <= 5 else 1
                    break
            ally_won = match['teams'][team]['win'] == "Win"
            data.loc[i, 'allyWonGame'] = ally_won
            data.loc[i + 1, 'allyWonGame'] = not ally_won
            found_match = True
            break
    if not found_match:
        print(f"Could not find match for row {i}")

Could not find match for row 1402
Could not find match for row 2118
Could not find match for row 2296
Server error 500, retrying in 60 s
Could not find match for row 2840
Server error 500, retrying in 60 s
Could not find match for row 3444
Could not find match for row 3518
Server error 503, retrying in 60 s
Could not find match for row 4140
Could not find match for row 4832
Server error 503, retrying in 60 s
Server error 500, retrying in 60 s
Could not find match for row 6634
Could not find match for row 6954
Could not find match for row 7272
Could not find match for row 7956
Could not find match for row 8762
Could not find match for row 9044
Could not find match for row 9196
Could not find match for row 9692
Server error 500, retrying in 60 s
Server error 500, retrying in 60 s
Could not find match for row 11060
Could not find match for row 11202
Could not find match for row 12308
Could not find match for row 12452
Could not find match for row 13030
Could not find match for row 13476
C

In [ ]:
data.to_csv('summoner_data.csv', index=False, header=True)

There are some data where the hours since the previous game are negative for some reason, so we use a different method to find them and fix the data.

In [ ]:
# def check_matchlist(begin_idx, team_idx, team, role, row):
#     teams = ['ally', 'enemy']
#     team_role = team + role
#     account_id = row[team_role + 'accountId']
# #     try:
#     match_list = make_riot_request(f"/lol/match/v3/matchlists/by-account/{account_id}", row['region'], 
#                                       params={'queue': [RANKED_QUEUE, NORMAL_QUEUE], 
#                                               'beginIndex': begin_idx})['matches']
#     print(len(match_list))
# #     except Exception as e:
# #         traceback.print_exc()
# #         print(f"Error when fetching matchlist for row {i}")
# #         return

#     summoner_id_set = {row[r + 'summonerId'] for r in team_roles}
#     for match_idx, match in enumerate(match_list):
#         print(match_idx, champ_id_to_name[match['champion']])
#         if match['queue'] != RANKED_QUEUE or row[team_role + 'championId'] != match['champion']:
#             continue
#         match_account_id_set = set()
#         try:
#             match = make_riot_request(f"/lol/match/v3/matches/{match['gameId']}", row['region'])
#         except Exception as e:
#             traceback.print_exc()
#             print(f"Could not fetch match for row {i}")
#             continue

#         match_summoner_id_set = set()
#         for p in match['participantIdentities']:
#             match_summoner_id_set.add(p['player']['summonerId'])
#             if p['player']['summonerId'] == row[team_role + "summonerId"]:
#                 match_team_idx = 0 if p['participantId'] <= 5 else 1
#         print(match_summoner_id_set)
#         print(summoner_id_set)
#         print(match_summoner_id_set == summoner_id_set)
#         print(match['gameId'])
#         if match_summoner_id_set != summoner_id_set:
#             continue

#         ally_won = match['teams'][match_team_idx]['win'] == "Win"
#         data.loc[i, 'allyWonGame'] = ally_won
#         data.loc[i + 1, 'allyWonGame'] = not ally_won

#         hours_since_last = (match_list[match_idx]['timestamp'] - match_list[match_idx + 1]['timestamp']) / (1000 * 60 * 60)
#         assert hours_since_last > 0
#         data.loc[i, team_role + 'hoursSincePreviousGame'] = hours_since_last
#         data.loc[i + 1, teams[1 - team_idx] + role + 'hoursSincePreviousGame'] = hours_since_last
#         return True
#     return False

# data = pd.read_csv('summoner_data.csv')
# teams = ['ally', 'enemy']
# team_roles = set()
# for team in teams:
#         for role in ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']:
#             team_roles.add(team + role)

# for i, row in data.iterrows():
#     if i % 2 == 1:
#         continue
#     if i != 2118:
#         continue
#     print(row['allyTOPsummonerName'], champ_id_to_name[row['allyTOPchampionId']], row['allyJUNGLEsummonerName'], row['region'])
#     for team_idx, team in enumerate(teams):
#         for role in ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']:
#             team_role = team + role
#             if row[team_role + 'hoursSincePreviousGame'] > 0 and row['allyWonGame'] != 'NaN':
#                 continue
#             found_match = check_matchlist(0, team_idx, team, role, row)
#             if not found_match:
#                 found_match = check_matchlist(99, team_idx, team, role, row)
#                 if not found_match:
#                     print(f"Could not find match in matchlist for row {i} for {team_role}")

ShigeNO Riven Senbonzakura BK6 euw1
20
0 Garen
1 Garen
2 Master Yi
3 Annie
4 Annie
5 Annie
6 Annie
7 Annie
8 Annie
9 Annie
10 Volibear
11 Sion
12 Tryndamere
13 Tryndamere
14 Riven
{25606592, 33091591, 76558185, 38680844, 86131149, 63687980, 106178452, 103699510, 47358456, 43019741}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3672925387
15 Xin Zhao
16 Ryze
17 Riven
{94559233, 67267906, 36652388, 40940933, 54366954, 96567755, 86131149, 30790253, 106339087, 23383310}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3672803377
18 Riven
{65450849, 70836961, 49654822, 83878378, 86131149, 81502899, 72669304, 99478137, 51786650, 92601245}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3672794844
19 Riven
{75401601, 43090692, 49209062, 77419274, 75412300, 86131149, 53234353, 52407605, 101398169, 58239519}
{54916929, 

{100497187, 94710181, 37827526, 105975912, 19532075, 90608399, 94571345, 87037650, 24761078, 68047607}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3661448402
38 Malzahar
{37827526, 93619783, 33424585, 32280490, 102689194, 60853616, 22463056, 81236630, 75147673, 51728285}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3660706579
39 Malzahar
{70517664, 77698628, 37827526, 82658566, 56670256, 81555313, 103377493, 82920213, 21834006, 91177150}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3660680934
40 Malzahar
{99578433, 40732066, 25919842, 104468454, 37827526, 55897576, 88529320, 97408438, 52545401, 61363194}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3660076258
41 Malzahar
{39978880, 93317984, 37827526, 79478217, 45949772, 40297200, 98697554, 

{87131137, 78070786, 105118530, 37827526, 88568199, 66852010, 20776906, 78088019, 21327734, 86291641}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3642760333
76 Malzahar
{49331585, 102420220, 37827526, 78908873, 22675571, 84568819, 72758931, 86291641, 67024666, 106287772}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3642758322
77 Malzahar
{21422529, 87897827, 37827526, 102968363, 102498575, 52019023, 30148791, 60709114, 53328795, 88131132}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3642715878
78 Malzahar
{52295489, 41649509, 37827526, 68674025, 34584754, 31100755, 25531539, 46607541, 61342165, 23554007}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3641872854
79 Malzahar
{81269664, 52688320, 42098947, 37827526, 62047178, 94787886, 105814193,

{56697987, 32289214, 37827526, 19525544, 108806829, 105805650, 51340117, 108560342, 28843289, 49358846}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3631963570
15 Malzahar
{86597122, 28905282, 37827526, 45126859, 22583660, 66574032, 108279188, 42881397, 107238429, 84778526}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3631942617
16 Malzahar
{37827526, 86830023, 44637639, 62047178, 72860426, 39654988, 94739053, 72377291, 50177615, 27096317}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3631530930
17 Malzahar
{106689184, 76397349, 37827526, 89976998, 70529897, 94288266, 53928621, 76641628, 58198141, 27837118}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3631507264
18 Malzahar
{92428675, 37827526, 40956907, 99079499, 92469682, 57279231, 83887006,

{42678752, 37827526, 19418439, 94774490, 94437709, 58161365, 46407543, 46685178, 80750107, 94463326}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3616292009
55 Malzahar
{94650018, 46653923, 37827526, 104717094, 81317318, 79158777, 104337005, 52458677, 52267897, 95099354}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3616247361
56 Malzahar
{37827526, 104608999, 70758760, 79850248, 85518383, 72758224, 108507828, 106467351, 61618360, 20170202}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3615956429
57 Malzahar
{105652354, 23184578, 32016836, 37827526, 36178023, 28527080, 33521165, 28966545, 80387187, 81501270}
{54916929, 46579937, 37827526, 62672490, 86131149, 92718577, 45409874, 40550643, 81504533, 86597846}
False
3615015338
58 Malzahar
{75540192, 37041634, 103849219, 37827526, 104548682, 65270858, 3195610

We have some data where the hours are negative. However, for some reason, some matches are missing from the matchlist, so we just delete that data.

# Data Cleanup

Let's remove some invalid data, and add some extra features.

In [73]:
data = pd.read_csv('summoner_data.csv')
valid_hours = pd.Series([True] * data.shape[0])
for team in ['ally', 'enemy']:
    for role in ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']:
        valid_hours &= data[team + role + 'hoursSincePreviousGame'] > 0
data = data[valid_hours]

In [49]:
# data = data.dropna()
# print(data.shape)

(5114, 432)


# Data Exploration

Let's look at distribution of regions.

In [27]:
data.groupby('region').count()['allyTOPsummonerId'].reset_index(name="count")

,region,count
0,euw1,4974
1,kr,5088
2,na1,4808


Let's look at distribution of ranks.

In [39]:
rank_cols = []
for team in ['ally', 'enemy']:
    rank_cols += [f"{team}{role}tier" for role in ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']]
rank_counts = data[rank_cols].stack().value_counts()
print(rank_counts / rank_counts.sum())

SILVER      0.489186
GOLD        0.307586
PLATINUM    0.103968
BRONZE      0.083914
DIAMOND     0.015212
MASTER      0.000134
dtype: float64


The rank distribution is a bit skewed towards higher ranks. Actual Bronze percentage is about 20%, and actual Gold and Platinum percentages are 21% and 5% respectively. However, this should be good enough.

# Deep Learning

In [40]:
from fastai.structured import *
from fastai.column_data import *

In [74]:
skip_col_names = ['summonerId', 'accountId', 'summonerName', 'championName']
skip_flds = []
for team in ['ally', 'enemy']:
    for role in ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']:
        skip_flds += [team + role + col for col in skip_col_names]
data = data.drop(columns=skip_flds)

In [79]:
cat_col_names = ['teamId', 'championId', 'role', 'hotStreak', 'veteran', 'freshBlood', 'tier', 'previousGameWon', 
                 'championDamageType']
cat_vars = ['region', 'allyWonGame']
for team in ['ally', 'enemy']:
    for role in ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']:
        cat_vars += [team + role + col for col in cat_col_names]
for v in cat_vars:
    data[v] = data[v].astype('category')

In [81]:
df, y, _, _ = proc_df(data, 'allyWonGame', do_scale=True)